In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType, ComplexType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, project, L2_norm, H1_norm
from utils.plotting import Mpl2DPlotter, Mpl2DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "./results/2dh-wq"
animator_dir = f'{save_dir}/animator'
Path(save_dir).mkdir(parents=True, exist_ok=True)
Path(animator_dir).mkdir(parents=True, exist_ok=True)

U_PATH = f"{save_dir}/uj.xdmf"
UD_PATH = f"{save_dir}/ujd.xdmf"
PRESSURE_PATH = f"{save_dir}/pressure.xdmf"

In [3]:
def generate_boundaries(points):
    return [(1, lambda x: np.isclose(x[0], points[0][0])),
            (2, lambda x: np.isclose(x[0], points[0][1])),
            (3, lambda x: np.isclose(x[1], points[1][0])),
            (4, lambda x: np.isclose(x[1], points[1][1])),]

In [4]:
def problem_setup(N, points, p, T, t, ct, dt, fluid, сontaminant, contamination_level, solver, pc):
    """
    Performs problem configuration w.r.t. given parameters
    """
    
    # Mesh and function space definition
    domain = mesh.create_rectangle(
        MPI.COMM_WORLD, 
        [(points[0][0], points[1][0]), (points[0][1], points[1][1])], 
        [N, N]
    )
    V = fem.VectorFunctionSpace(domain, ("CG", 2))

    u = TrialFunction(V)
    v = TestFunction(V)
    
    # Definition functions of physical characteristics
    w_ro, w_c, w_eta = fluids.loc[fluid, ['Density', 'Speed of sound', 'Viscosity']]
    c_ro, c_c, c_eta = fluids.loc[сontaminant, ['Density', 'Speed of sound', 'Viscosity']]
    
    ro, c, eta = [(1.0-contamination_level)*vals[0] + contamination_level*vals[1] 
                  for vals in [(w_ro, c_ro), (w_c, c_c), (w_eta, c_eta)]]
        
    # Construction of problem form
    GAMMA, BETA = 0.5, 0.5
    
    mm = ScalarType(ro) * inner(u, v) * dx
    aa = ScalarType(ro * c**2) * inner(grad(u), grad(v)) * dx
    cc = ScalarType(4./3 * eta) * inner(grad(u), grad(v)) * dx
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt**2 * BETA * aa)
    
    # Definition of boundary conditions
    boundaries = generate_boundaries(points)
    measure = generate_boundary_measure(boundaries, domain)
    
    u_D = lambda x: [x[0] * 0.0, x[1] * 0.0]
    u_N1 = fem.Constant(domain, ScalarType(p))
    u_N2 = fem.Constant(domain, ScalarType((0, 0)))

    bcs = [BoundaryCondition("Dirichlet", 1, u_D, V, u, v, measure).bc,
           BoundaryCondition("Dirichlet", 3, u_D, V, u, v, measure).bc, 
           BoundaryCondition("Dirichlet", 4, u_D, V, u, v, measure).bc]
     
    nbcs = [BoundaryCondition("Neumann", 2, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 2, u_N2, V, u, v, measure).bc] 
    
    return {
        'Params': (N, points, fluid, ro, c, T, dt, t, ct, GAMMA, BETA, solver, pc),
        'FunctionSpace': (domain, V, u, v),
        'Forms': (mm, aa, cc), 
        'Problem': (F, bcs, nbcs)
    }

In [5]:
def solve_problem(config):
    N, points, fluid, ro, c, T, dt, t_stop, ct, GAMMA, BETA, solver_type, pc = config['Params']
    domain, V, u, v = config['FunctionSpace']
    mm, aa, cc = config['Forms']
    F, bcs, nbcs = config['Problem']
    
    xdmf_u = io.XDMFFile(domain.comm, U_PATH, "w")
    xdmf_ud = io.XDMFFile(domain.comm, UD_PATH, "w")
    xdmf_p = io.XDMFFile(domain.comm, PRESSURE_PATH, "w")
    xdmf_u.write_mesh(domain)
    xdmf_ud.write_mesh(domain)
    xdmf_p.write_mesh(domain)
    
    # Create initial condition
    initial_condition = lambda x: [x[0] * 0.0, x[1] * 0.0]
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_d_tg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_d_tg.interpolate(initial_condition)
    
    p = project(ro * c**2 * div(uj), domain, ("CG", 1))
    
    # Construct the right hand side of the problem
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(solver_type.lower())
    solver.getPC().setType(pc.lower())

    # Solve problem at each time step
    num_steps = int(T / dt)
    t = 0.0
    
    time_data = {
        't'   : [0.0,],
        'uj'  : [np.copy(uj.x.array[:]),],
        'uj_d': [np.copy(uj_d.x.array[:]),],
        'p'   : [np.copy(p.x.array[:]),]
    }
    
    uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
    L1 = nbcs[0] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L2 = nbcs[1] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    linear_form_1 = fem.form(rhs(L1))
    linear_form_2 = fem.form(rhs(L2))
    
    b1 = fem.petsc.create_vector(linear_form_1)
    b2 = fem.petsc.create_vector(linear_form_2)
    
    has_norm = False
    L2_n, H1_n = 0.0, 0.0
    
    for i in range(num_steps):
        t += dt

        # Multiply values of uj_d by dt and GAMMA, 
        # because form does not support muliplication by constant
        uj_d_d_tg.x.array[:] = uj_d.x.array * dt * GAMMA

        b = b1 if t < t_stop else b2
        linear_form = linear_form_1 if t < t_stop else linear_form_2
        
        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # Apply Dirichlet boundary condition to the vector
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, bcs)

        # Solve linear problem
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()

        # Update solution
        uj1_d_array = uj_d.x.array + dt * uj_dd.x.array

        uj.x.array[:] = uj.x.array + 0.5 * dt * (uj_d.x.array + uj1_d_array)
        uj_d.x.array[:] = uj1_d_array

        # Save results into the files
        if i % 5 == 0:
            p = project(ro * c**2 * div(uj), domain, ("CG", 1))

            xdmf_u.write_function(uj, t)
            xdmf_ud.write_function(uj_d, t)
            xdmf_p.write_function(p, t)
            
            if i % 25 == 0:
                time_data['t'].append(t)
                time_data['uj'].append(np.copy(uj.x.array))
                time_data['uj_d'].append(np.copy(uj_d.x.array))
                time_data['p'].append(np.copy(p.x.array))
        
        if not has_norm and t >= ct:
            L2_n = L2_norm(uj)
            H1_n = H1_norm(uj)

            has_norm = True
        
    xdmf_u.close()
    xdmf_ud.close()
    xdmf_p.close()
    
    return L2_n, H1_n, uj, uj_d, p, time_data

#### Available fluids

In [6]:
fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
fluids

,Density,Speed of sound,Viscosity,Heat,Thermal conductivity,Thermal expansion
Fluid,,,,,,
Water,1000,1500,0.000894,4182.0,0.598,0.00015
Fuel oil,890,1360,2.022000,2090.0,0.150,0.00070
Oil,760,1470,0.000500,NaN,NaN,NaN
Glycerin,1260,1905,1.500000,NaN,NaN,NaN


In [7]:
config_1 = problem_setup(
    N=100,
    points=[[0.0, 0.02], [0.0, 0.02]],
    p=(-1.0, 0.0),
    T=5.0e-5,
    t=1.5e-6,
    ct=2.0e-5,
    dt=5.0e-8,
    fluid='Water',
    сontaminant='Fuel oil',
    contamination_level=0.0,
    solver='CG',
    pc='ASM'
)
config_2 = problem_setup(
    N=100,
    points=[[0.0, 0.02], [0.0, 0.02]],
    p=(-1.0, 0.0),
    T=5.0e-5,
    t=1.5e-6,
    ct=2.0e-5,
    dt=5.0e-8,
    fluid='Water',
    сontaminant='Fuel oil',
    contamination_level=0.35,
    solver='CG',
    pc='ASM'
)

In [8]:
L2_n_1, H1_n_1, uj_1, uj_d_1, p_1, time_data_1 = solve_problem(config_1)
L2_n_2, H1_n_2, uj_2, uj_d_2, p_2, time_data_2 = solve_problem(config_2)
N, points, *_ = config_1['Params']

In [15]:
def data_diff(data_1, data_2):
    return list(arr_1-arr_2 for arr_1, arr_2 in zip(data_1, data_2))

In [16]:
mpa = Mpl2DAnimator(layout=[['1', '2', '3'], 
                            ['4', '5', '6'], 
                            ['7', '8', '9']], time=time_data_1['t'], projection='2d')
mpa.update_figure(figsize=(16, 12), fontsize=20)

mpa.add_fun(fun=uj_1, time_data=time_data_1['uj'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='1', color='r', title='Displacement(O)')
mpa.add_fun(fun=uj_2, time_data=time_data_2['uj'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='2', color='r', title='Displacement(C)')
mpa.add_fun(fun=uj_1, time_data=data_diff(time_data_1['uj'], time_data_2['uj']), type='real', project='mag',
            points=points, N=[25, 25], axes_id='3', color='r', title='Displacement(D)')

mpa.add_fun(fun=uj_d_1, time_data=time_data_1['uj_d'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='4', color='g', title='Velocity(O)')
mpa.add_fun(fun=uj_d_2, time_data=time_data_2['uj_d'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='5', color='g', title='Velocity(C)')
mpa.add_fun(fun=uj_d_1, time_data=data_diff(time_data_1['uj_d'], time_data_2['uj_d']), type='real', project='mag',
            points=points, N=[25, 25], axes_id='6', color='g', title='Velocity(D)')

mpa.add_fun(fun=p_1, time_data=time_data_1['p'], type='real', project='scalar',
            points=points, N=[25, 25], axes_id='7',  color='b', title='Pressure(O)')
mpa.add_fun(fun=p_2, time_data=time_data_2['p'], type='real', project='scalar',
            points=points, N=[25, 25], axes_id='8',  color='b', title='Pressure(C)')
mpa.add_fun(fun=p_1, time_data=data_diff(time_data_1['p'], time_data_2['p']), type='real', project='scalar',
            points=points, N=[25, 25], axes_id='9',  color='b', title='Pressure(D)')

mpa.write(path=animator_dir, filename='animation_2d.mp4', fps=10, interval=2000)

In [17]:
Video(f'{animator_dir}/animation_2d.mp4', width=800)

In [18]:
mpa = Mpl2DAnimator(layout=[['1', '2', '3'], 
                            ['4', '5', '6'], 
                            ['7', '8', '9']], time=time_data_1['t'], projection='3d')
mpa.update_figure(figsize=(16, 12), fontsize=20)

mpa.add_fun(fun=uj_1, time_data=time_data_1['uj'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='1', color='r', title='Displacement(O)')
mpa.add_fun(fun=uj_2, time_data=time_data_2['uj'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='2', color='r', title='Displacement(C)')
mpa.add_fun(fun=uj_1, time_data=data_diff(time_data_1['uj'], time_data_2['uj']), type='real', project='mag',
            points=points, N=[25, 25], axes_id='3', color='r', title='Displacement(D)')

mpa.add_fun(fun=uj_d_1, time_data=time_data_1['uj_d'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='4', color='g', title='Velocity(O)')
mpa.add_fun(fun=uj_d_2, time_data=time_data_2['uj_d'], type='real', project='mag',
            points=points, N=[25, 25], axes_id='5', color='g', title='Velocity(C)')
mpa.add_fun(fun=uj_d_1, time_data=data_diff(time_data_1['uj_d'], time_data_2['uj_d']), type='real', project='mag',
            points=points, N=[25, 25], axes_id='6', color='g', title='Velocity(D)')

mpa.add_fun(fun=p_1, time_data=time_data_1['p'], type='real', project='scalar',
            points=points, N=[25, 25], axes_id='7',  color='b', title='Pressure(O)')
mpa.add_fun(fun=p_2, time_data=time_data_2['p'], type='real', project='scalar',
            points=points, N=[25, 25], axes_id='8',  color='b', title='Pressure(C)')
mpa.add_fun(fun=p_1, time_data=data_diff(time_data_1['p'], time_data_2['p']), type='real', project='scalar',
            points=points, N=[25, 25], axes_id='9',  color='b', title='Pressure(D)')

mpa.write(path=animator_dir, filename='animation_3d.mp4', fps=10, interval=2000)

In [19]:
Video(f'{animator_dir}/animation_3d.mp4', width=800)